In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import pickle
from app.util.timer import Timer
from app.util.Differ import Differ
from main import YoloRuntimeTest
import time
from threading import Thread

CPU input

In [2]:
args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image",
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx_runtime_model = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "onnxruntime_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino_model = {
    "weights": "./app/weights/yolov9c_openvino_model/yolov9c.xml", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "openvino_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

Initilize YOLO runtime test and Timer classes

In [3]:
yolo_runtime_test = YoloRuntimeTest()
timer = Timer()

In [5]:
timer.start()
cpu_pytorch_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_pytorch)
timer.stop()
print(f'Execution function time: {timer.elapsed_time} s')
print(cpu_pytorch_ultralytics_image)


[INFO] Inference Image


[[[    0.27451     0.28627     0.27843 ...     0.23137     0.23529     0.22745]
  [    0.28235     0.27843     0.28627 ...     0.23137     0.22745     0.22745]
  [    0.28627      0.2902      0.2902 ...     0.24314     0.23922     0.22745]
  ...
  [    0.19216         0.2     0.20392 ...     0.15294     0.14118     0.13333]
  [    0.19216     0.21176         0.2 ...     0.15294     0.14118     0.13333]
  [        0.2     0.20392     0.20392 ...     0.14902     0.14118     0.13333]]

 [[    0.56471     0.57647     0.56863 ...     0.54118     0.54118     0.53333]
  [    0.57255     0.56863     0.57647 ...     0.53725     0.53333     0.53333]
  [    0.57647     0.58039     0.58039 ...     0.54902      0.5451     0.53333]
  ...
  [    0.29412     0.30196     0.30588 ...     0.26275     0.25098     0.24314]
  [    0.29412      0.3098     0.30196 ...     0.26275     0.25098     0.24314]
  [    0.30196     0.30588     0.30588 ...     0.25882     0.25098     0.24314]]


In [ ]:
pwd

In [ ]:
timer.start()
cpu_openvino_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_openvino)
timer.stop()
print(f'Execution function time: {timer.elapsed_time} s')
print(cpu_openvino_ultralytics_image)

In [ ]:
timer.start()
cpu_openvino_model_image = yolo_runtime_test.openvino_run_image(args_openvino_model)
timer.stop()
print(f'Execution function time: {timer.elapsed_time} s')
print(cpu_openvino_model_image)

In [6]:
timer.start()
cpu_onnx_ultralytics_image = yolo_runtime_test.ultralytics_run_image(args_onnx)
timer.stop()
print(f'Execution function time: {timer.elapsed_time} s')
print(cpu_onnx_ultralytics_image)

[INFO] Inference Image
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...


[[[    0.27451     0.28627     0.27843 ...     0.23137     0.23529     0.22745]
  [    0.28235     0.27843     0.28627 ...     0.23137     0.22745     0.22745]
  [    0.28627      0.2902      0.2902 ...     0.24314     0.23922     0.22745]
  ...
  [    0.19216         0.2     0.20392 ...     0.15294     0.14118     0.13333]
  [    0.19216     0.21176         0.2 ...     0.15294     0.14118     0.13333]
  [        0.2     0.20392     0.20392 ...     0.14902     0.14118     0.13333]]

 [[    0.56471     0.57647     0.56863 ...     0.54118     0.54118     0.53333]
  [    0.57255     0.56863     0.57647 ...     0.53725     0.53333     0.53333]
  [    0.57647     0.58039     0.58039 ...     0.54902      0.5451     0.53333]
  ...
  [    0.29412     0.30196     0.30588 ...     0.26275     0.25098     0.24314]
  [    0.29412      0.3098     0.30

In [ ]:
timer.stop()

In [7]:
timer.start()
cpu_onnx_runtime_model_image = yolo_runtime_test.onnxruntime_run_image(args_onnx_runtime_model)
timer.stop()
print(f'Execution function time: {timer.elapsed_time} s')
print(cpu_onnx_runtime_model_image)

[INFO] Inference Image
[[[    0.27451     0.28627     0.27843 ...     0.23137     0.23529     0.22745]
  [    0.28235     0.27843     0.28627 ...     0.23137     0.22745     0.22745]
  [    0.28627      0.2902      0.2902 ...     0.24314     0.23922     0.22745]
  ...
  [    0.19216         0.2     0.20392 ...     0.15294     0.14118     0.13333]
  [    0.19216     0.21176         0.2 ...     0.15294     0.14118     0.13333]
  [        0.2     0.20392     0.20392 ...     0.14902     0.14118     0.13333]]

 [[    0.56471     0.57647     0.56863 ...     0.54118     0.54118     0.53333]
  [    0.57255     0.56863     0.57647 ...     0.53725     0.53333     0.53333]
  [    0.57647     0.58039     0.58039 ...     0.54902      0.5451     0.53333]
  ...
  [    0.29412     0.30196     0.30588 ...     0.26275     0.25098     0.24314]
  [    0.29412      0.3098     0.30196 ...     0.26275     0.25098     0.24314]
  [    0.30196     0.30588     0.30588 ...     0.25882     0.25098     0.24314]]

 

Difference CPU

In [ ]:
pd.set_option('display.expand_frame_repr', False)

def generate_difference_df(image1, image2, label):
    differ = Differ(np.array(image1), np.array(image2))
    result = differ.find_difference()
    return pd.DataFrame(result, columns=[label, "cpu conf_diff", "cpu box_diff (px)"])

df_pt_openvino_ultralytics = generate_difference_df(cpu_pytorch_ultralytics_image[0], cpu_openvino_ultralytics_image[0], "pt vs openvino+ultralytics")
df_pt_openvino_model = generate_difference_df(cpu_pytorch_ultralytics_image[0], cpu_openvino_model_image[0], "pt vs openvino model")
df_pt_onnx_ultralytics = generate_difference_df(cpu_pytorch_ultralytics_image[0], cpu_onnx_ultralytics_image[0], "pt vs onnx+ultralytics")
df_pt_onnxruntime_model = generate_difference_df(cpu_pytorch_ultralytics_image[0], cpu_onnx_runtime_model_image[0], "pt vs onnxruntime model")

df_combined = pd.concat([df_pt_openvino_ultralytics, df_pt_openvino_model, df_pt_onnx_ultralytics, df_pt_onnxruntime_model], axis=1)

print(df_combined)

Average CPU Time (10)

In [ ]:
result_time = []
ultralytics_inference_time = []

In [ ]:
def run_inference_with_args(inference_func, args):
    def wrapper():
        return inference_func(args)
    return wrapper

def collect_execution_times(run_inference_func, args, iterations=100):
    execution_times = []
    for i in range(iterations):
        args["source"] = f"./app/assets/sample_image_{i}.jpg"
        wrapper_func = run_inference_with_args(run_inference_func, args)
        result = wrapper_func()
        execution_times.append(result[1] * 1000)
        ultralytics_inference_time.append(result[2])
    return execution_times

args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image",
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx_runtime_model = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "onnxruntime_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "ultralytics",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino_model = {
    "weights": "./app/weights/yolov9c_openvino_model/yolov9c.xml", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "inference_type": "openvino_model",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_pytorch))
# result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))
# result_time.append(collect_execution_times(yolo_runtime_test.openvino_run_image, args_openvino_model))
# result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))
# result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx_runtime_model))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.openvino_run_image, args_openvino_model))

In [ ]:
result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx_runtime_model))

In [ ]:
result_time = np.array(result_time)
df = pd.DataFrame(np.transpose(result_time), 
                  columns=["pytorch+ultralytics time cpu (ms)",
                           "openvino+ultralytics time cpu (ms)",
                           "onnx​+ultralytics time cpu (ms)",
                           "openvino model time cpu (ms)", 
                           "onnx runtime model time cpu (ms)"])
df.describe(percentiles=[.9, .95])

In [ ]:
ultralytics_inference_time

In [ ]:
def split_list_into_arrays(input_list):
    list1 = input_list[:100]
    list2 = input_list[100:200]
    list3 = input_list[200:300]
    
    return list1, list2, list3

array1, array2, array3 = split_list_into_arrays(ultralytics_inference_time)
infer_timer_list = [array1, array2, array3]

print(infer_timer_list)

infer_timer_list = np.array(infer_timer_list)
df_infer = pd.DataFrame(np.transpose(infer_timer_list), columns=["pytorch+ultralytics built-in profiler time cpu (ms)", "openvino+ultralytics built-in profiler time cpu (ms)", "onnx+ultralytics built-in profiler time cpu (ms)"])
df_infer.describe(percentiles=[.9, .95])

Save CPU result

In [ ]:
with open('./app/saved_pkl/cpu_df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('./app/saved_pkl/cpu_infer_df.pkl', 'wb') as f:
    pickle.dump(df_infer, f)